In [23]:
# Advent of Code 2023
# Day 5 Problem 1

with open("aoc_05_input.txt") as f:
    A = f.read().strip().split("\n\n")

# c+p test case here
TEST = """seeds: 79 14 55 13

seed-to-soil map:
50 98 2
52 50 48

soil-to-fertilizer map:
0 15 37
37 52 2
39 0 15

fertilizer-to-water map:
49 53 8
0 11 42
42 0 7
57 7 4

water-to-light map:
88 18 7
18 25 70

light-to-temperature map:
45 77 23
81 45 19
68 64 13

temperature-to-humidity map:
0 69 1
1 0 69

humidity-to-location map:
60 56 37
56 93 4""".strip().split("\n\n")

for i, line in enumerate(TEST):
    print(i, line)


0 seeds: 79 14 55 13
1 seed-to-soil map:
50 98 2
52 50 48
2 soil-to-fertilizer map:
0 15 37
37 52 2
39 0 15
3 fertilizer-to-water map:
49 53 8
0 11 42
42 0 7
57 7 4
4 water-to-light map:
88 18 7
18 25 70
5 light-to-temperature map:
45 77 23
81 45 19
68 64 13
6 temperature-to-humidity map:
0 69 1
1 0 69
7 humidity-to-location map:
60 56 37
56 93 4


In [24]:
# problem helper functions here

def giveSeeds(inp):
    seeds = [int(x) for x in str(inp[0]).split(": ")[1].split(" ")]
    return seeds

def yieldMapping(map, n):
    #print(map)
    M = map.split("\n")
    for p in M[1:]:
        d, s, r = p.split(" ")
        a,b,c = int(d), int(s), int(r) 
        if n in range(b, b + c):
            delta = b + c - n 
            return n - b + a
    return n

def p1(lines):
    seeds = giveSeeds(lines)
    cand = []
    for seed in seeds:
        for map in lines[1:]:
            #print(f"{seed}->", end="")
            seed = yieldMapping(map, seed)
        #print(f"{seed}")
        #print()
        cand.append(seed)
    return min(cand)


In [25]:
# expected output: 13
print(f"  Test:\t{p1(TEST)}")
p1_ans = p1(A)
print(f"Part 1:\t{p1_ans}")


  Test:	35
Part 1:	551761867


In [26]:
# Part 2
def seedRanges(inp):
    S = [int(x) for x in str(inp[0]).split(": ")[1].split(" ")]
    res = []
    for i in range(len(S)//2):
        # Use clopen sets of the form [a,b)
        res.append([S[2*i], S[(2*i)+1]])
    return res


In [27]:
def getAlmanac(lines):
    # Please give me an array of sorted [D S R] for each map type
    maps_as_given = [sorted([[int(x) for x in p.split(" ")] for p in line.strip().split("\n")[1:]]) for line in lines]
    for map in maps_as_given:
        if map[0][0]:   
            map.insert(0, [0, 0, map[0][0] - 1])
    maps = [[[a, a+c, b, b+c] for a, b, c in map] for map in maps_as_given]
    return maps[::-1]

def pClopen(sR):
    print("Seed ranges: (half-open sets of the form [a,b)")
    for r in sR:
        print(r, end=" -> ")
        print(f"[{r[0]}, {r[0] + r[1]})")

def yieldClopens(seed_ranges):
    return [[s[0], s[0]+s[1]] for s in seed_ranges]

def p2_debug(lines):
    sR = sorted(seedRanges(lines))
    pClopen(sR)
    seeds = yieldClopens(sR)
    print()
    almanac = getAlmanac(lines[1:])
    for row in almanac:
        print(row)
    print()
    # Good up til here
    SEARCH_MIN = 1
    SEARCH_MAX = 1_000_000
    print(f"Searching between {SEARCH_MIN}-{SEARCH_MAX}")
    for n in range(SEARCH_MIN, SEARCH_MAX):
        if not n % 10_000: 
            print(n)
        loc_cand = n
        found = False
        for i,map in enumerate(almanac):
            found = False
            for a_min, a_max, b_min, b_max in map:
                if not found and a_min <= loc_cand < a_max:
                    delta = b_min - a_min
                    loc_cand += delta
                    found = True
        if any([loc_cand in range(S[0], S[1]) for S in seeds]):
            return n
    return 0

def p2(lines, search_min, search_max):
    sR = sorted(seedRanges(lines))
    seeds = yieldClopens(sR)
    almanac = getAlmanac(lines[1:])
    print(f"Searching between {search_min}-{search_max}")
    for n in range(search_min, search_max):
        #if not n % 1_000_000:  print(n)
        loc_cand = n
        found = False
        for i,map in enumerate(almanac):
            found = False
            for a_min, a_max, b_min, b_max in map:
                if not found and a_min <= loc_cand < a_max:
                    delta = b_min - a_min
                    loc_cand += delta
                    found = True
        if any([loc_cand in range(S[0], S[1]) for S in seeds]):
            return n
    return 0


In [28]:
print(p2(TEST,1,100))


Searching between 1-100
46


In [29]:
# up to 30_000_000 took 5 minutes.
# need to come back and make this more efficient lol
search_floor = 55_000_000
print(p2(A, search_floor, p1_ans))


Searching between 55000000-551761867
57451709
